In [1]:
#Step (1): Read Data
import pandas as pd #Read data
import numpy as np  #Array

# Read in dataset (encoding = 'Big5'可顯示出繁體中文)
data_train = pd.read_excel("01train_fix.xlsx")
data_test = pd.read_excel("01test.xlsx")
data_out = pd.read_excel("01test.xlsx")
data_train.head() # Show data (看看是否讀取成功)

,手機螢幕溫度(平均),手機螢幕溫度(最大),手機背蓋溫度(平均),平均APP運作指標/小時,充電時間指標,手機電池電量,CPU跑分階級,手機外殼/包膜指標,外型平均曲率,上網頻率指標,通話頻率指標,新機/二手/老舊,內容容量比率,CPU效能等級,Underclocking
0,-2.600452,-2.833071,-3.553067,-0.984729,-0.820966,-5.324059,3,-0.741141,3.593824,-1.395510,-1.225014,-0.494159,0.750427,-1.955921,0
1,-2.606880,-2.838397,-3.544191,-0.984776,2.337184,-5.283197,4,-0.603315,4.295257,-1.395792,-1.225314,-0.494159,0.479721,-1.463128,1
2,-2.628558,-2.857238,-3.556987,-0.984526,0.042620,-5.313652,3,-0.286121,3.844972,-1.395733,-1.225263,-0.494159,0.754607,-1.955921,0
3,-2.627114,-2.852910,-3.541104,-0.984609,-0.586660,-5.333278,3,-0.070108,3.763350,-1.395879,-1.225406,-0.494159,0.306494,-1.955921,1
4,-2.618020,-2.845929,-3.547520,-0.984577,-0.669485,-5.323154,3,-0.448527,3.619669,-1.395727,-1.225276,-0.494159,0.237039,-1.955921,0


In [2]:
#drop 上網試試看,心肌、平均app不drop
columns_to_drop = ['外型平均曲率', '手機外殼/包膜指標', '手機螢幕溫度(最大)', 'CPU效能等級', '平均APP運作指標/小時', '新機/二手/老舊']
data_train.drop(columns=columns_to_drop, axis=1, inplace=True)
data_test.drop(['id'] + columns_to_drop, axis=1, inplace=True)

In [3]:
from sklearn.preprocessing import MinMaxScaler


columns_to_normalize = [ '手機螢幕溫度(平均)', '手機背蓋溫度(平均)','手機電池電量', 'CPU跑分階級','內容容量比率','充電時間指標', '通話頻率指標', '上網頻率指標']

# 初始化標準化物件
min_max_scaler = MinMaxScaler()

# 將資料標準化
data_train[columns_to_normalize] = min_max_scaler.fit_transform(data_train[columns_to_normalize])
data_test[columns_to_normalize] = min_max_scaler.transform(data_test[columns_to_normalize])

# 顯示標準化後的資料
print(data_train.head())

   手機螢幕溫度(平均)  手機背蓋溫度(平均)    充電時間指標    手機電池電量   CPU跑分階級    上網頻率指標    通話頻率指標  \
0    0.751415    0.351942  0.040835  0.256490  0.000000  0.741359  0.741666   
1    0.656482    0.480422  0.758026  0.834848  0.333333  0.208022  0.174929   
2    0.336291    0.295218  0.236949  0.403789  0.000000  0.318551  0.270737   
3    0.357618    0.525096  0.094044  0.125995  0.000000  0.043074  0.000000   
4    0.491931    0.432228  0.075235  0.269297  0.000000  0.330148  0.246156   

     內容容量比率  Underclocking  
0  0.999200              0  
1  0.947380              1  
2  1.000000              0  
3  0.914220              1  
4  0.900924              0  


In [4]:
data_train.describe()

,手機螢幕溫度(平均),手機背蓋溫度(平均),充電時間指標,手機電池電量,CPU跑分階級,上網頻率指標,通話頻率指標,內容容量比率,Underclocking
count,3801.000000,3801.000000,3801.000000,3801.000000,3801.000000,3801.000000,3801.000000,3801.000000,3801.000000
mean,0.458673,0.427200,0.232764,0.523468,0.100149,0.295079,0.262878,0.856439,0.429098
std,0.214805,0.122263,0.239493,0.268746,0.224448,0.209245,0.214012,0.182481,0.495012
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.304877,0.354648,0.030426,0.285566,0.000000,0.110583,0.068397,0.794055,0.000000
50%,0.456419,0.437134,0.147783,0.550710,0.000000,0.262961,0.226719,0.933536,0.000000
75%,0.623431,0.503336,0.384236,0.741779,0.000000,0.450491,0.414328,0.986253,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
data_test.describe()

,手機螢幕溫度(平均),手機背蓋溫度(平均),充電時間指標,手機電池電量,CPU跑分階級,上網頻率指標,通話頻率指標,內容容量比率
count,1777.000000,1777.000000,1777.000000,1777.000000,1777.000000,1.777000e+03,1.777000e+03,1777.000000
mean,620.859722,801.201224,0.278881,1139.362829,0.107672,4.868339e+06,4.215554e+06,1.020854
std,219.432104,209.489179,0.051571,199.101581,0.225472,3.485699e+06,3.438235e+06,0.034615
min,161.706418,88.869339,0.227270,747.582679,0.000000,4.188546e+03,2.320196e+03,0.855549
25%,460.595785,681.578837,0.234756,964.607635,0.000000,1.901596e+06,1.173945e+06,1.010487
50%,625.740884,823.035533,0.262646,1163.733237,0.000000,4.351049e+06,3.637668e+06,1.034171
75%,786.101087,934.504131,0.312183,1298.790688,0.000000,7.287323e+06,6.595174e+06,1.044316
max,1156.054888,1760.644475,0.446792,1489.760393,1.000000,1.647474e+07,1.605086e+07,1.046975


In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler  #建構standardization的物件

#name_list_outlier = ['手機螢幕溫度(平均)','手機螢幕溫度(最大)','手機背蓋溫度(平均)','平均APP運作指標/小時','充電時間指標',
                #'手機電池電量','手機外殼/包膜指標','外型平均曲率','上網頻率指標',
                #'通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

#name_list_outlier = ['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','充電時間指標','手機電池電量','通話頻率指標',
                        #'新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

name_list_outlier =[ '手機背蓋溫度(平均)',  '內容容量比率']
name_pre_outlier = ['Mobile Screen Temperature (Max)','Mobile Phone Back Cover Temperature (Average)','Average APP Operation Index/Hour','Mobile Phone Battery Level','Content Capacity Ratio']


#離群值處理
def  outlier_del(data_frame,name_list_outlier,n1=5,n2=5):
    for i, name in enumerate(name_list_outlier):
        print(name)
        print("Shape Of The Before Ouliers: ", data_frame[name].shape)
    
        # 计算IQR
        Q1 = np.percentile(data_frame[name], 25)
        Q3 = np.percentile(data_frame[name], 75)
        IQR = Q3 - Q1

        # 定义离群值范围
        lower_bound = Q1 - n1 * IQR
        upper_bound = Q3 + n2 * IQR

        # 过滤离群值
        filtered_entries = ((data_frame[name] >= lower_bound) & (data_frame[name] <= upper_bound))
        data_frame = data_frame[filtered_entries]
    
        print("Shape Of The After Ouliers: ",data_frame[name].shape)
        print('===================================================================================================================')
    return data_frame

In [7]:
class_distribution = data_train['Underclocking'].value_counts() #指定data中的label欄位，進行各類別數量的統計
print("Class Distribution:")
print(class_distribution)

Class Distribution:
Underclocking
0    2170
1    1631
Name: count, dtype: int64


In [8]:
feature_ex =['手機螢幕溫度(平均)', '手機背蓋溫度(平均)', '充電時間指標', '手機電池電量', 'CPU跑分階級', '通話頻率指標', '內容容量比率', '上網頻率指標']
x_data = data_train[feature_ex]
y_data = data_train['Underclocking']

In [9]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=3333, min_samples_leaf=2,min_samples_split=8,max_depth=6,oob_score=True,random_state=42,n_jobs=-1)

# 定义 KFold 交叉验证对象
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 使用 cross_validate 进行交叉验证
cv_results = cross_validate(rf_model, x_data, y_data, cv=kfold, scoring='accuracy', return_train_score=True)

# 输出每个 fold 的训练得分和验证得分
for i in range(5):
    train_score = cv_results['train_score'][i]
    test_score = cv_results['test_score'][i]
    print(f"Fold {i+1} - Training Score: {train_score}, Validation Score: {test_score}")

# 输出平均训练得分和平均验证得分
mean_train_score = np.mean(cv_results['train_score'])
mean_test_score = np.mean(cv_results['test_score'])
print(f"\nMean Training Score: {mean_train_score}")
print(f"Mean Validation Score: {mean_test_score}")

Fold 1 - Training Score: 0.7588815789473684, Validation Score: 0.7148488830486203
Fold 2 - Training Score: 0.7530417625780993, Validation Score: 0.7039473684210527
Fold 3 - Training Score: 0.7540282801709964, Validation Score: 0.7026315789473684
Fold 4 - Training Score: 0.7546859585662611, Validation Score: 0.7026315789473684
Fold 5 - Training Score: 0.7530417625780993, Validation Score: 0.7144736842105263

Mean Training Score: 0.754735868568165
Mean Validation Score: 0.7077066187149873


In [10]:
#Step (4): 把dataset分成訓練跟驗證
from sklearn.model_selection import train_test_split
#test_size=0.3 為比例-> training:test=7:3
trainX, testX, trainY, testY = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

RandomForestClassifier()
rf_model = RandomForestClassifier(n_estimators=3333, min_samples_leaf=2,min_samples_split=8,max_depth=6,oob_score=True,random_state=42,n_jobs=-1)

# 使用模型对训练集进行训练
rf_model.fit(trainX, trainY)

# 获取训练集和验证集的预测结果
Y_train_pred = rf_model.predict(trainX)
Y_test_pred = rf_model.predict(testX)

# 计算训练集和验证集的准确率
train_accuracy = accuracy_score(trainY, Y_train_pred)
test_accuracy = accuracy_score(testY, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

print('特徵重要程度: ',rf_model.feature_importances_)

Training Accuracy: 0.756578947368421
Validation Accuracy: 0.7187910643889619
特徵重要程度:  [0.10597786 0.1566704  0.0422482  0.15435503 0.02743946 0.07030154
 0.36002669 0.08298081]


In [12]:
predicted_ans = rf_model.predict(data_test[feature_ex])
my_submission = pd.DataFrame({'id': data_out.id, 'Underclocking': predicted_ans})
my_submission.to_csv('submission.csv', index=False)
